In [163]:
import numpy as np
import pandas as pd
import json
from rauth import OAuth1Service,OAuth1Session
import requests 
import webbrowser
import datetime
# from requests_oauthlib import OAuth1
import time
import os
import matplotlib.pyplot as plt


consumer_key='85bdd7b968b11348fd8682f33a1d1c82'
consumer_secret='1fd4678dd9db8eefeb9046bfcfda129d'

is_long = False
is_short = False

etrade = OAuth1Service(
    name="etrade",
    consumer_key='85bdd7b968b11348fd8682f33a1d1c82',
    consumer_secret='1fd4678dd9db8eefeb9046bfcfda129d',
    request_token_url="https://api.etrade.com/oauth/request_token",
    access_token_url="https://api.etrade.com/oauth/access_token",
    authorize_url="https://us.etrade.com/e/t/etws/authorize?key={}&token={}",
    base_url="https://api.etrade.com")

request_token, request_token_secret = etrade.get_request_token(
        params={"oauth_callback": "oob", "format": "json"})

authorize_url = etrade.authorize_url.format(etrade.consumer_key, request_token)
webbrowser.open(authorize_url)
text_code = input("Please accept agreement and enter text code from browser: ")
session = etrade.get_auth_session(request_token,
                                  request_token_secret,
                                  params={"oauth_verifier": text_code})
exp_day = str(input('Enter Expiration Day'))
exp_month = str(input('Enter Expiration Month'))

accounts_list_url = "https://api.etrade.com/v1/accounts/list.json"
r = session.get(accounts_list_url)
# r = requests.get(accounts_list_url,auth=auth)
r = json.loads(r.text)
rollover_key = r['AccountListResponse']['Accounts']['Account'][2]['accountIdKey']
def get_data():
    
    r = session.get('https://api.etrade.com/v1/market/quote/SPY.json')
    r = json.loads(r.content)
    spy_market_price = round((r['QuoteResponse']['QuoteData'][0]['All']['ask']+r['QuoteResponse']['QuoteData'][0]['All']['bid'])/2,2)
    curr_time = int(str(datetime.datetime.now().month).zfill(2)+str(datetime.datetime.now().day).zfill(2)+str(datetime.datetime.now().hour).zfill(2)+str(datetime.datetime.now().minute).zfill(2)+str(datetime.datetime.now().second).zfill(2))
    return spy_market_price,curr_time


def list_orders():
    r = session.get(f'https://api.etrade.com/v1/accounts/{rollover_key}/orders.json')
    r = json.loads(r.content)
    last_order_status = r['OrdersResponse']['Order'][0]['OrderDetail'][0]['status']
    return last_order_status
    
def cancel_order():
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    r = session.get(f'https://api.etrade.com/v1/accounts/{rollover_key}/orders.json')
    cancel_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/cancel.json'
    r = json.loads(r.text)
    cancel_oid = str(r['OrdersResponse']['Order'][0]['orderId'])
    payload = '''<CancelOrderRequest>
                   <orderId>{}</orderId>
                </CancelOrderRequest>'''
    payload = payload.format(cancel_oid)
    r = session.put(cancel_url,header_auth=True,headers=headers,data=payload)
    print(payload)
    print(cancel_url)
    return r.content

def balances():
    r = session.get(f'https://api.etrade.com/v1/accounts/{rollover_key}/portfolio.json')
    r = json.loads(r.content)
    return len(r['PortfolioResponse']['AccountPortfolio'][0]['Position'])

def buy_open_order(limit_price):
    preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
    client_order_id = str(np.random.randint(1000000000, 2000000000))
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    payload = """<PreviewOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>LIMIT</priceType>
                           <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice>{}</limitPrice>
                           <Instrument>
                               <Product>
                                   <callPut>CALL</callPut>
                                   <expiryDay>{}</expiryDay>
                                   <expiryMonth>{}</expiryMonth>
                                   <expiryYear>2022</expiryYear>
                                   <securityType>OPTN</securityType>
                                   <strikePrice>{}</strikePrice>
                                   <symbol>SPY</symbol>
                               </Product>
                               <orderAction>BUY_OPEN</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>1</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

    payload = payload.format(client_order_id,limit_price,exp_day,exp_month,strike_price)
    r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)
    if r.status_code == 200 :
        r = json.loads(r.content)
        if r['PreviewOrderResponse']['totalOrderValue'] < r['PreviewOrderResponse']['cashBpDetails']['settled']['currentBp']:
            preview_id = r['PreviewOrderResponse']['PreviewIds'][0]['previewId']
            place_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/place.json'
            payload = """<PlaceOrderRequest>
                           <orderType>OPTN</orderType>
                           <clientOrderId>{}</clientOrderId>
                           <PreviewIds>
                              <previewId>{}</previewId>
                           </PreviewIds>
                           <Order>
                              <allOrNone>false</allOrNone>
                                  <priceType>LIMIT</priceType>
                                  <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                                  <marketSession>REGULAR</marketSession>
                                  <stopPrice></stopPrice>
                                  <limitPrice>{}</limitPrice>
                              <Instrument>
                                 <Product>
                                     <callPut>CALL</callPut>
                                     <expiryDay>{}</expiryDay>
                                     <expiryMonth>{}</expiryMonth>
                                     <expiryYear>2022</expiryYear>
                                     <securityType>OPTN</securityType>
                                     <strikePrice>{}</strikePrice>
                                     <symbol>SPY</symbol>
                                 </Product>
                                 <orderAction>BUY_OPEN</orderAction>
                                 <quantityType>QUANTITY</quantityType>
                                 <quantity>1</quantity>
                              </Instrument>
                           </Order>
                        </PlaceOrderRequest>"""

            payload = payload.format(client_order_id,preview_id,limit_price,exp_day,exp_month,strike_price)
            r = session.post(place_order_url,header_auth=True,headers=headers,data=payload)
            if r.status_code == 200:
                print('SPY Long Executed!')
        else:
            print('SPY Long Place order failed')
    else:
        print('SPY Long Preview order failed')

def sell_close_order(prev_strike_price,order_price):
    limit_price = order_price + .12
    preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
    client_order_id = str(np.random.randint(1000000000, 2000000000))
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    payload = """<PreviewOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>LIMIT</priceType>
                           <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice>{}</limitPrice>
                           <Instrument>
                               <Product>
                                   <callPut>CALL</callPut>
                                   <expiryDay>{}</expiryDay>
                                   <expiryMonth>{}</expiryMonth>
                                   <expiryYear>2022</expiryYear>
                                   <securityType>OPTN</securityType>
                                   <strikePrice>{}</strikePrice>
                                   <symbol>SPY</symbol>
                               </Product>
                               <orderAction>SELL_CLOSE</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>1</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

    payload = payload.format(client_order_id,limit_price,exp_day,exp_month,prev_strike_price)
    r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)
    if r.status_code == 200:
        r = json.loads(r.content)
        preview_id = r['PreviewOrderResponse']['PreviewIds'][0]['previewId']
        place_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/place.json'
        payload = """<PlaceOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <PreviewIds>
                          <previewId>{}</previewId>
                       </PreviewIds>
                       <Order>
                          <allOrNone>false</allOrNone>
                              <priceType>LIMIT</priceType>
                              <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                              <marketSession>REGULAR</marketSession>
                              <stopPrice></stopPrice>
                              <limitPrice>{}</limitPrice>
                          <Instrument>
                             <Product>
                                 <callPut>CALL</callPut>
                                 <expiryDay>{}</expiryDay>
                                 <expiryMonth>{}</expiryMonth>
                                 <expiryYear>2022</expiryYear>
                                 <securityType>OPTN</securityType>
                                 <strikePrice>{}</strikePrice>
                                 <symbol>SPY</symbol>
                             </Product>
                             <orderAction>SELL_CLOSE</orderAction>
                             <quantityType>QUANTITY</quantityType>
                             <quantity>1</quantity>
                          </Instrument>
                       </Order>
                    </PlaceOrderRequest>"""

        payload = payload.format(client_order_id,preview_id,limit_price,exp_day,exp_month,prev_strike_price)
        r = session.post(place_order_url,header_auth=True,headers=headers,data=payload)
        if r.status_code == 200:
            print('SPY Long Closed!')
        else:
            print('SPY Close Place order failed - trying again')
            sell_close_order(prev_strike_price)
    else:
        print('SPY Close Preview order failed - trying again')
        sell_close_order(prev_strike_price)
        
def buy_open_order_put(limit_price):
    preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
    client_order_id = str(np.random.randint(1000000000, 2000000000))
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    payload = """<PreviewOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>LIMIT</priceType>
                           <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice>{}</limitPrice>
                           <Instrument>
                               <Product>
                                   <callPut>PUT</callPut>
                                   <expiryDay>{}</expiryDay>
                                   <expiryMonth>{}</expiryMonth>
                                   <expiryYear>2022</expiryYear>
                                   <securityType>OPTN</securityType>
                                   <strikePrice>{}</strikePrice>
                                   <symbol>SPY</symbol>
                               </Product>
                               <orderAction>BUY_OPEN</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>1</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

    payload = payload.format(client_order_id,limit_price,exp_day,exp_month,put_strike_price)
    r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)
    if r.status_code == 200 :
        r = json.loads(r.content)
        if r['PreviewOrderResponse']['totalOrderValue'] < r['PreviewOrderResponse']['cashBpDetails']['settled']['currentBp']:
            preview_id = r['PreviewOrderResponse']['PreviewIds'][0]['previewId']
            place_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/place.json'
            payload = """<PlaceOrderRequest>
                           <orderType>OPTN</orderType>
                           <clientOrderId>{}</clientOrderId>
                           <PreviewIds>
                              <previewId>{}</previewId>
                           </PreviewIds>
                           <Order>
                              <allOrNone>false</allOrNone>
                                  <priceType>LIMIT</priceType>
                                  <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                                  <marketSession>REGULAR</marketSession>
                                  <stopPrice></stopPrice>
                                  <limitPrice>{}</limitPrice>
                              <Instrument>
                                 <Product>
                                     <callPut>PUT</callPut>
                                     <expiryDay>{}</expiryDay>
                                     <expiryMonth>{}</expiryMonth>
                                     <expiryYear>2022</expiryYear>
                                     <securityType>OPTN</securityType>
                                     <strikePrice>{}</strikePrice>
                                     <symbol>SPY</symbol>
                                 </Product>
                                 <orderAction>BUY_OPEN</orderAction>
                                 <quantityType>QUANTITY</quantityType>
                                 <quantity>1</quantity>
                              </Instrument>
                           </Order>
                        </PlaceOrderRequest>"""

            payload = payload.format(client_order_id,preview_id,limit_price,exp_day,exp_month,put_strike_price)
            r = session.post(place_order_url,header_auth=True,headers=headers,data=payload)
            if r.status_code == 200:
                print('SPY Short Executed!')
        else:
            print('SPY Short Place order failed')
    else:
        print('SPY Short Preview order failed')

def sell_close_order_put(prev_strike_price,order_price):
    limit_price = order_price + .12
    preview_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/preview.json'
    client_order_id = str(np.random.randint(1000000000, 2000000000))
    headers = {"Content-Type": "application/xml", "consumerKey": consumer_key}
    payload = """<PreviewOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <Order>
                           <allOrNone>false</allOrNone>
                           <priceType>LIMIT</priceType>
                           <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                           <marketSession>REGULAR</marketSession>
                           <stopPrice></stopPrice>
                           <limitPrice>{}</limitPrice>
                           <Instrument>
                               <Product>
                                   <callPut>PUT</callPut>
                                   <expiryDay>{}</expiryDay>
                                   <expiryMonth>{}</expiryMonth>
                                   <expiryYear>2022</expiryYear>
                                   <securityType>OPTN</securityType>
                                   <strikePrice>{}</strikePrice>
                                   <symbol>SPY</symbol>
                               </Product>
                               <orderAction>SELL_CLOSE</orderAction>
                               <quantityType>QUANTITY</quantityType>
                               <quantity>1</quantity>
                           </Instrument>
                       </Order>
                   </PreviewOrderRequest>"""

    payload = payload.format(client_order_id,limit_price,exp_day,exp_month,prev_strike_price)
    r = session.post(preview_order_url,header_auth=True,headers=headers,data=payload)
    if r.status_code == 200:
        r = json.loads(r.content)
        preview_id = r['PreviewOrderResponse']['PreviewIds'][0]['previewId']
        place_order_url = f'https://api.etrade.com/v1/accounts/{rollover_key}/orders/place.json'
        payload = """<PlaceOrderRequest>
                       <orderType>OPTN</orderType>
                       <clientOrderId>{}</clientOrderId>
                       <PreviewIds>
                          <previewId>{}</previewId>
                       </PreviewIds>
                       <Order>
                          <allOrNone>false</allOrNone>
                              <priceType>LIMIT</priceType>
                              <orderTerm>GOOD_UNTIL_CANCEL</orderTerm>
                              <marketSession>REGULAR</marketSession>
                              <stopPrice></stopPrice>
                              <limitPrice>{}</limitPrice>
                          <Instrument>
                             <Product>
                                 <callPut>PUT</callPut>
                                 <expiryDay>{}</expiryDay>
                                 <expiryMonth>{}</expiryMonth>
                                 <expiryYear>2022</expiryYear>
                                 <securityType>OPTN</securityType>
                                 <strikePrice>{}</strikePrice>
                                 <symbol>SPY</symbol>
                             </Product>
                             <orderAction>SELL_CLOSE</orderAction>
                             <quantityType>QUANTITY</quantityType>
                             <quantity>1</quantity>
                          </Instrument>
                       </Order>
                    </PlaceOrderRequest>"""

        payload = payload.format(client_order_id,preview_id,limit_price,exp_day,exp_month,prev_strike_price)
        r = session.post(place_order_url,header_auth=True,headers=headers,data=payload)
        if r.status_code == 200:
            print('SPY Short Closed!')
        else:
            print('SPY Close Place order failed - trying again')
            sell_close_order(prev_strike_price)
    else:
        print('SPY Close Preview order failed - trying again')
        sell_close_order(prev_strike_price)
        
        
spy_data_dict = {'times':[],'prices':[]}  

Please accept agreement and enter text code from browser: 78S69
Enter Expiration Day15
Enter Expiration Month6


In [ ]:
second = datetime.datetime.now().second

hour = datetime.datetime.now().hour
minute = datetime.datetime.now().minute
print(hour)

try_long = False
try_short = False
long = False
short = False

while hour < 8 :
    if second == 0:
        time.sleep(1)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
    else:
        time.sleep(0.2)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute

print('between 8 and 8:30')

while hour > 7 and hour < 9 and minute < 30:
    if second == 0:
        time.sleep(1)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
    else:
        time.sleep(0.2)
        second = datetime.datetime.now().second
        hour = datetime.datetime.now().hour
        minute = datetime.datetime.now().minute
last_minute = minute - 1
print('8:30')

def nearest_resistance(r_price):
    nearest_whole = round(r_price,0)
    if nearest_whole < r_price:
        return float(nearest_whole) + 1
    else:
        return float(nearest_whole) + .50
    
def nearest_support(r_price):
    nearest_whole = round(r_price,0)
    if nearest_whole > r_price:
        return float(nearest_whole) - 1
    else:
        return float(nearest_whole) -.50    
    
def get_option(month,day,side,r_price):
    
    r = session.get(f'https://api.etrade.com/v1/market/quote/SPY:2022:{month}:{day}:{side}:{r_price}.json')
    r = json.loads(r.content)
    return round(float(r['QuoteResponse']['QuoteData'][0]['All']['bid']),2),round(float(r['QuoteResponse']['QuoteData'][0]['All']['ask']),2)

while hour > 7 and hour < 15:

    open_position = balances()
    order_status = list_orders()
    
#     if try_long and open_position > 1:
#         sell_close_order(prev_strike_price,option_order_price)
#         try_long = False
        
#     if try_short and open_position > 1:
#         sell_close_order_put(prev_strike_price,option_order_price)
        
    
    try:
        spy_price,curr_time = get_data()
    except:
        try:
            spy_price,curr_time = get_spy_data()
        except:
            print('could not get price')
            pass
        
    if long:
        bid,ask = get_option(exp_month,exp_day,'CALL',prev_strike_price)
        sell_price = float(bid)-.1
        if option_order_price - bid >= .70:
            long = False
            print(f'Stop Long SPY @', spy_price, bid)
#             sell_close_order(prev_strike_price,sell_price)
#             
            
        
    if short:
        bid,ask = get_option(exp_month,exp_day,'PUT',prev_strike_price)
        sell_price = float(bid)-.1
        if option_order_price - bid >= .70:
            short = False
            print('Stop Short SPY @', spy_price, bid)
#             sell_close_order_put(prev_strike_price,sell_price)
            

    if (try_long or try_short) and (datetime.datetime.now()-try_time).total_seconds()/60 > 14:
#         cancel_order()
        try_long = False
        try_short = False
        
    if try_long:
        bid,ask = get_option(exp_month,exp_day,'CALL',prev_strike_price)
        if ask <= option_order_price:
            long = True
            try_long = False
            trade_time = datetime.datetime.now()
            print(f'SPY Long @', spy_order_price, option_order_price)
        
    if try_short:
        bid,ask = get_option(exp_month,exp_day,'PUT',prev_strike_price)
        if ask <= option_order_price:
            short = True
            try_short = False
            trade_time = datetime.datetime.now()
            print(f'SPY Short @', spy_order_price, option_order_price)
        
    if long:
        bid,ask = get_option(exp_month,exp_day,'CALL',prev_strike_price)
        if bid - option_order_price >= .12:
            long = False
            print(f'Close Long SPY @', spy_price, bid)
        
    if short:
        bid,ask = get_option(exp_month,exp_day,'PUT',prev_strike_price)
        if bid - option_order_price >= .12:
            short = False
            print('Close Short SPY @', spy_price)
    
#     if open_position > 0 and order_status != 'OPEN' and long:
#         sell_close_order(prev_strike_price,option_order_price)
        
#     if open_position > 0 and order_status != 'OPEN' and short:
#         sell_close_order_put(prev_strike_price,option_order_price)

    if minute != last_minute and minute%5 == 0:
        
        
        last_minute = datetime.datetime.now().minute

        strike_price = str(int(round(spy_price,0)))
        spy_data_dict['times'].append(curr_time)
        spy_data_dict['prices'].append(spy_price)
        spy_df = pd.DataFrame({'time':spy_data_dict['times'],'price':spy_data_dict['prices']})
        spy_df.set_index('time',inplace=True)
        spy_df['change'] = spy_df['price'] - spy_df['price'].shift(1)
        spy_df['max_change'] = spy_df['change'].rolling(3).max()
        spy_df['min_change'] = spy_df['change'].rolling(3).min()
        resist = nearest_resistance(spy_price)
        support = nearest_support(spy_price)
        print(spy_df.iloc[-1]['price'],spy_df.iloc[-1]['max_change'],spy_df.iloc[-1]['min_change'],resist,support)
        
        if hour == 8 and minute == 30:
            time.sleep(5)
            
        else:
             
            if not long and not short and not try_long and not try_short and spy_df.iloc[-1]['min_change'] > 0 and len(spy_df)> 2:
                bid, ask = get_option(exp_month,exp_day,'PUT',strike_price)
                prev_strike_price = strike_price
                spy_order_price = resist
                option_order_price = round(bid - ((resist - spy_price) * .50),2)
                try_short = True
                try_time = datetime.datetime.now()
                print('Spy Price short trigger:',spy_order_price,'option order price',option_order_price)
#                 buy_open_order_put(option_order_price)
                
            if not long and not short and not try_long and not try_short and spy_df.iloc[-1]['max_change'] < 0 and len(spy_df)> 2:
                bid, ask = get_option(exp_month,exp_day,'CALL',strike_price)
                prev_strike_price = strike_price
                spy_order_price = support
                option_order_price = round(bid - ((spy_price-support) * .50),2)
                try_long = True
                try_time = datetime.datetime.now()
                print('Spy Price long trigger:',spy_order_price,'option order price',option_order_price)
#                 buy_open_order(option_order_price)
                
                

            time.sleep(5)
            second = datetime.datetime.now().second
            minute = datetime.datetime.now().minute
            hour = datetime.datetime.now().hour

            
    else:
        try:
            spy_price,curr_time = get_data()
        except:
            try:
                spy_price,curr_time = get_spy_data()
            except:
                pass
        
            
        time.sleep(5)
        second = datetime.datetime.now().second
        minute = datetime.datetime.now().minute
        hour = datetime.datetime.now().hour
        
        
            

8
between 8 and 8:30
8:30
378.47 nan nan 379.0 377.5
378.76 nan nan 379.5 378.0
378.29 nan nan 379.0 377.5
378.63 0.339999999999975 -0.46999999999997044 379.5 378.0
379.05 0.4200000000000159 -0.46999999999997044 380.0 378.5
379.14 0.4200000000000159 0.08999999999997499 380.0 378.5
Spy Price short trigger: 380.0 option order price 3.0
378.57 0.4200000000000159 -0.5699999999999932 379.5 378.0
378.51 0.08999999999997499 -0.5699999999999932 379.5 378.0
378.46 -0.05000000000001137 -0.5699999999999932 379.0 377.5
378.58 0.12000000000000455 -0.060000000000002274 379.5 378.0
378.04 0.12000000000000455 -0.5399999999999636 379.0 377.5
377.73 0.12000000000000455 -0.5399999999999636 378.5 377.0
377.47 -0.2599999999999909 -0.5399999999999636 378.0 376.5
Spy Price long trigger: 376.5 option order price 3.15
377.38 -0.09000000000003183 -0.3100000000000023 378.0 376.5
SPY Long @ 376.5 3.15
376.56 -0.09000000000003183 -0.8199999999999932 377.5 376.0
Close Long SPY @ 376.85 3.3
377.25 0.6899999999999977